In [2]:
import librosa 
import librosa.display as dsp
import soundfile as sf
import numpy as np
import scipy
from scipy.fft import fft, fftfreq
import matplotlib.pyplot as plt 
import plotly
import plotly.graph_objs as go
from IPython.display import Audio as audioPlayer

In [3]:
#functions for plotting in time and freq domain
def plotFourierMag(song_mag_data):
    time = np.arange(0,len(song))/sr
    fig, ax = plt.subplots()
    ax.plot(song_frequency,song_mag_data)
    ax.set(xlabel='Time(s)',ylabel='sound amplitude')
    plt.show()
def plotInTime():
    time = np.arange(0,len(song))/sr
    fig, ax = plt.subplots()
    ax.plot(time,song)
    ax.set(xlabel='Time(s)',ylabel='sound amplitude')
    plt.show()
def plotWhatever(y):
    x_axis = np.arange(0,y.size)
    fig, ax = plt.subplots()
    ax.plot(x_axis,y)
    ax.set(xlabel='X',ylabel='Y')
    plt.show()

In [4]:
song, sr = librosa.load("antonio_vivaldi_4seasons_winter.wav",offset=35.0,duration=20.0)
# song, sr = librosa.load("fallout nv_Slow Sax.wav",offset=1.0 ,duration=20.0)
print(song)

C:\Users\I1bra\anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


FileNotFoundError: [Errno 2] No such file or directory: 'antonio_vivaldi_4seasons_winter.wav'

In [ ]:
audioPlayer(data=song, rate=sr, autoplay=False)
# audioPlayer("antonio_vivaldi_4seasons_winter.wav", autoplay=True)

In [ ]:
# decompose the signal using rfft
def fourier_function(song):
    complex_fft = scipy.fft.rfft(song)
    magnitude = np.abs(complex_fft)
    phase = np.angle(complex_fft)
    frequency = scipy.fft.rfftfreq(len(song), 1 / sr)
    return magnitude, phase, frequency

song_magnitude, song_phase, song_frequency=fourier_function(song)
print(song_frequency.shape)

In [ ]:
plotFourierMag(song_magnitude)

In [ ]:
slider1_ranges= [[200,1500],[50,100]]
viola = [[1500,3500]]
bass = [[50,150]]
#Take a copy of the magnitude list, then do operations
new_mag = song_magnitude.copy()
def equalizerRange_Window(magnitude,frequency,slider_ranges,slider_dB): 
    #manipulate mag over a specific range
    for rng in slider_ranges:
        # adjust magnitude of range in list      
        target_freq= list(np.where((frequency>rng[0])&(frequency<rng[1]))[0])
        for i in target_freq:
            new_mag[i]=song_magnitude[i]*10**(slider_dB)
        print(rng[0])

In [ ]:
#Take a copy of the magnitude list, then do operations
new_mag = song_magnitude.copy()
# function that uses triangle window to manipulate data, still not done
def equalizerRange_Triangle(magnitude,frequency,slider_ranges,slider_dB): 
    #manipulate mag over a specific range
    for rng in slider_ranges:
        # adjust magnitude of range in list      
        target_freq= list(np.where((frequency>rng[0])&(frequency<rng[1]))[0])
        #generate a trianglular window
        triangle_window=10**((slider_dB)*scipy.signal.windows.triang(len(target_freq)))
        #triangle_window=slider_dB*scipy.signal.windows.triang(len(target_freq))
        for i,window in zip(target_freq,triangle_window):
            new_mag[i]=song_magnitude[i]*window
        print(rng[0])

In [ ]:
# inverse reconstruction of signal ==> y[frequency]= (magnitude*(e^(phase)))
def calc_inv_fourier(mag, phase):
    New_signal = np.multiply(mag, np.exp(1j * phase))
    inv_fourier_signal = np.real(scipy.fft.irfft(New_signal))
    #inv_fourier_signal = inv_fourier_signal.astype(np.int16) #commented cuz it breaks the function, dunno why
    return inv_fourier_signal 


In [ ]:
# function calls for adjusting the curve according to the value and range list of each slider
equalizerRange_Triangle(song_magnitude,song_frequency,viola,1.5)
equalizerRange_Triangle(song_magnitude,song_frequency,bass,1.5)
# equalizerRange_Window(song_magnitude,song_frequency,viola,2)
# equalizerRange_Window(song_magnitude,song_frequency,bass,1.5)

plotFourierMag(new_mag)
song_edited = calc_inv_fourier(new_mag, song_phase)


In [ ]:
# audioPlayer(data=y_harmonic, rate=sr, autoplay=True)
# play the edited data
audioPlayer(data=song_edited, rate=sr, autoplay=False)

In [ ]:
# for writing the song 
sf.write('after.wav', song_edited, sr)

In [ ]:
# extract the percussive component
y_percussive = librosa.effects.percussive(song, margin=5)
y_harmonic = librosa.effects.harmonic(song, margin=5)

In [ ]:
# drawing a spectrogram
# d = librosa.stft(y_percussive)
# D = librosa.amplitude_to_db(np.abs(d),ref=np.max)
# fig,ax = plt.subplots(2,1,sharex=True,figsize=(10,10))
# img = dsp.specshow(D, y_axis='linear', x_axis='s',sr=sr,ax=ax[0])
# ax[0].set(title='Linear frequency power spectrogram')
# ax[0].label_outer()
# dsp.specshow(D,y_axis='log',x_axis='s',sr=sr,ax=ax[1])
# ax[1].set(title='Log frequency power spectrogram')
# ax[1].label_outer()
# fig.colorbar(img, ax=ax, format='%+2.f dB')

In [ ]:
# Bin separation and operations
# 125 Hz–1KHz freq range for violins
# 260 Hz–3.5KHz for celeste
# slider_ranges= [[[200,1500],[1500,3500]]]

# def equalizer(magnitude,frequency,sliders_value): 
# # iterate on slider values 
#     for itr in sliders_value:
#         #for each slider, iterate on the ranges
#         for comp in slider_ranges:
#             #for each range (which is a list)
#             for rng in comp:
#                 # adjust magnitude of range in list      
#                 target_freq= list(np.where((frequency>rng[0])&(frequency<rng[1]))[0])
#                 for i in target_freq:
#                     song_magnitude[i]=song_magnitude[i]*10**(itr)                
# equalizer(song_magnitude,song_frequency,[0,0,0])

In [ ]:
# generating a trianglular window
# triangle_window=(10**(slider_dB))*scipy.signal.windows.triang(len(target_freq))
triangle_window=10*scipy.signal.windows.triang(100)
plotWhatever(triangle_window)
print(triangle_window)
# for i in triangle_window:
#     print(i)

In [ ]:
print("you got this")

In [ ]:
# plotting part with plotly, abit slow
# xpoints = np.arange(0, song.size, 1)
# ypoints = song
# trace0 = go.Scatter(
#    x = xpoints, y = ypoints
# )
# data = [trace0]
# plotly.offline.iplot({ "data": data,"layout": go.Layout(title="Sine wave")})